In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
import math
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn import metrics

seed=10
random.seed(seed)
np.random.seed(seed)


In [ ]:
!wget https://www.dropbox.com/s/aqdapb8c1w6su9c/RUL_FD001.txt
!wget https://www.dropbox.com/s/n0xnhkvifjx61i2/test_FD001.txt
!wget https://www.dropbox.com/s/xc1nkz6gnia2ptr/train_FD001.txt

In [ ]:
dir_path = ''
train_file = 'train_FD001.txt'
test_file = 'test_FD001.txt'
index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i+1) for i in range(0,21)]
col_names = index_names + setting_names + sensor_names
train = pd.read_csv((dir_path+train_file), sep='\s+', header=None, names=col_names)
test = pd.read_csv((dir_path+test_file), sep='\s+', header=None, names=col_names)
y_test = pd.read_csv((dir_path+'RUL_FD001.txt'), sep='\s+', header=None, names=['RemainingUsefulLife'])

print(train.shape)
train

(20631, 26)


,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,21.61,551.43,2388.19,9065.52,1.3,48.07,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,21.61,550.86,2388.23,9065.11,1.3,48.04,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,21.61,550.94,2388.24,9065.90,1.3,48.09,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,21.61,550.68,2388.25,9073.72,1.3,48.39,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


In [ ]:
train[index_names].describe()
train[index_names].groupby('unit_nr').max().describe()

,time_cycles
count,100.000000
mean,206.310000
std,46.342749
min,128.000000
25%,177.000000
50%,199.000000
75%,229.250000
max,362.000000


In [ ]:
train[setting_names].describe()

,setting_1,setting_2,setting_3
count,20631.000000,20631.000000,20631.0
mean,-0.000009,0.000002,100.0
std,0.002187,0.000293,0.0
min,-0.008700,-0.000600,100.0
25%,-0.001500,-0.000200,100.0
50%,0.000000,0.000000,100.0
75%,0.001500,0.000300,100.0
max,0.008700,0.000600,100.0


In [ ]:
train[sensor_names].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
s_1,20631.0,518.670000,6.537152e-11,518.6700,518.6700,518.6700,518.6700,518.6700
s_2,20631.0,642.680934,5.000533e-01,641.2100,642.3250,642.6400,643.0000,644.5300
s_3,20631.0,1590.523119,6.131150e+00,1571.0400,1586.2600,1590.1000,1594.3800,1616.9100
s_4,20631.0,1408.933782,9.000605e+00,1382.2500,1402.3600,1408.0400,1414.5550,1441.4900
s_5,20631.0,14.620000,3.394700e-12,14.6200,14.6200,14.6200,14.6200,14.6200
s_6,20631.0,21.609803,1.388985e-03,21.6000,21.6100,21.6100,21.6100,21.6100
s_7,20631.0,553.367711,8.850923e-01,549.8500,552.8100,553.4400,554.0100,556.0600
s_8,20631.0,2388.096652,7.098548e-02,2387.9000,2388.0500,2388.0900,2388.1400,2388.5600
s_9,20631.0,9065.242941,2.208288e+01,9021.7300,9053.1000,9060.6600,9069.4200,9244.5900
s_10,20631.0,1.300000,4.660829e-13,1.3000,1.3000,1.3000,1.3000,1.3000


In [ ]:
y_test

,RemainingUsefulLife
0,112
1,98
2,69
3,82
4,91
...,...
95,137
96,82
97,59
98,117


In [ ]:
train['cycle_norm'] = train['time_cycles']
cols_normalize = train.columns.difference(['id','time_cycles','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train[cols_normalize]), columns=cols_normalize, index=train.index)
join_df = train[train.columns.difference(cols_normalize)].join(norm_train_df)
train = join_df.reindex(columns = train.columns)

train

,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,cycle_norm
0,0.0,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,0.726248,0.242424,0.109755,0.0,0.369048,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,0.000000
1,0.0,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,0.628019,0.212121,0.100242,0.0,0.380952,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,0.002770
2,0.0,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,0.710145,0.272727,0.140043,0.0,0.250000,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,0.005540
3,0.0,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,0.740741,0.318182,0.124518,0.0,0.166667,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,0.008310
4,0.0,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,0.668277,0.242424,0.149960,0.0,0.255952,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,0.011080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,1.0,196,0.477011,0.250000,0.0,0.0,0.686747,0.587312,0.782917,0.0,1.0,0.254428,0.439394,0.196491,0.0,0.726190,0.170576,0.558824,0.194344,0.656791,0.0,0.750000,0.0,0.0,0.271318,0.109500,0.540166
20627,1.0,197,0.408046,0.083333,0.0,0.0,0.701807,0.729453,0.866475,0.0,1.0,0.162641,0.500000,0.194651,0.0,0.708333,0.211087,0.500000,0.188668,0.727203,0.0,0.583333,0.0,0.0,0.124031,0.366197,0.542936
20628,1.0,198,0.522989,0.500000,0.0,0.0,0.665663,0.684979,0.775321,0.0,1.0,0.175523,0.515152,0.198196,0.0,0.738095,0.281450,0.529412,0.212148,0.922278,0.0,0.833333,0.0,0.0,0.232558,0.053991,0.545706
20629,1.0,199,0.436782,0.750000,0.0,0.0,0.608434,0.746021,0.747468,0.0,1.0,0.133655,0.530303,0.233285,0.0,0.916667,0.208955,0.514706,0.203065,0.823394,0.0,0.583333,0.0,0.0,0.116279,0.234466,0.548476


In [ ]:
def add_RUL(df):
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["time_cycles"].max()
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)   
    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame["time_cycles"]
    result_frame["RUL"] = remaining_useful_life
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame
  
train = add_RUL(train)
train[index_names+['RUL']]

,unit_nr,time_cycles,RUL
0,0.0,1,191
1,0.0,2,190
2,0.0,3,189
3,0.0,4,188
4,0.0,5,187
...,...,...,...
20626,1.0,196,4
20627,1.0,197,3
20628,1.0,198,2
20629,1.0,199,1


In [ ]:
train.apply(pd.to_numeric)
list=[]
for i in range(0,20631):
  if train["RUL"][i]<50:
    list.append(1)
  elif train["RUL"][i]<100:
    list.append(2)
  elif train["RUL"][i]<150:
    list.append(3)
  else:
    list.append(4)

train["health"]=list
  

In [ ]:
train.drop('s_1', inplace=True, axis=1)
train.drop('s_5', inplace=True, axis=1)
train.drop('s_6', inplace=True, axis=1)
train.drop('s_10', inplace=True, axis=1)
train.drop('s_16', inplace=True, axis=1)
train.drop('s_18', inplace=True, axis=1)
train.drop('s_19', inplace=True, axis=1)
train.drop('unit_nr', inplace=True, axis=1)
train.drop('setting_3', inplace=True, axis=1)
train.drop('time_cycles', inplace=True, axis=1)

In [ ]:
curr_max = 190
train.apply(pd.to_numeric)
for i in range(0,20629):
  if train["RUL"][i]==0:
    curr_max = math.floor((train["RUL"][i+1])/10)*10
  if train["RUL"][i]>=curr_max:
    train.drop(i, inplace=True, axis=0)


train

,setting_1,setting_2,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21,cycle_norm,RUL,health
2,0.252874,0.750000,0.343373,0.369523,0.370527,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375,0.005540,189,4
3,0.540230,0.500000,0.343373,0.256159,0.331195,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386,0.008310,188,4
4,0.390805,0.333333,0.349398,0.257467,0.404625,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502,0.011080,187,4
5,0.252874,0.416667,0.268072,0.292784,0.272113,0.776167,0.181818,0.125415,0.184524,0.637527,0.220588,0.169832,0.330512,0.250000,0.651163,0.652720,0.013850,186,4
6,0.557471,0.583333,0.382530,0.463920,0.261985,0.723027,0.181818,0.167818,0.303571,0.773987,0.220588,0.167097,0.278953,0.333333,0.744186,0.667219,0.016620,185,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,0.477011,0.250000,0.686747,0.587312,0.782917,0.254428,0.439394,0.196491,0.726190,0.170576,0.558824,0.194344,0.656791,0.750000,0.271318,0.109500,0.540166,4,1
20627,0.408046,0.083333,0.701807,0.729453,0.866475,0.162641,0.500000,0.194651,0.708333,0.211087,0.500000,0.188668,0.727203,0.583333,0.124031,0.366197,0.542936,3,1
20628,0.522989,0.500000,0.665663,0.684979,0.775321,0.175523,0.515152,0.198196,0.738095,0.281450,0.529412,0.212148,0.922278,0.833333,0.232558,0.053991,0.545706,2,1
20629,0.436782,0.750000,0.608434,0.746021,0.747468,0.133655,0.530303,0.233285,0.916667,0.208955,0.514706,0.203065,0.823394,0.583333,0.116279,0.234466,0.548476,1,1


In [ ]:
new_index = []
for i in range(1,20061):
  new_index.append(i)

In [ ]:
#train.set_index('new')
s = pd.Series(new_index)
train = train.set_index([s])

In [ ]:
train["health"]

1        4
2        4
3        4
4        4
5        4
        ..
20056    1
20057    1
20058    1
20059    1
20060    1
Name: health, Length: 20060, dtype: int64

In [ ]:
for i in range(0,2006):
  curr = 0
  for j in range(1,11):
    curr = curr+(train["health"][(10*i)+j]) 
  curr = round(curr/10)
  for j in range(1,11):
    train["health"][(10*i)+j]=curr  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
train.drop('RUL', inplace=True, axis=1)

In [ ]:
train.shape

(20060, 18)

In [ ]:
y_train=train['health']
train.drop('health', inplace=True, axis=1)

In [ ]:
train

,setting_1,setting_2,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21,cycle_norm
1,0.252874,0.750000,0.343373,0.369523,0.370527,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375,0.005540
2,0.540230,0.500000,0.343373,0.256159,0.331195,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386,0.008310
3,0.390805,0.333333,0.349398,0.257467,0.404625,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502,0.011080
4,0.252874,0.416667,0.268072,0.292784,0.272113,0.776167,0.181818,0.125415,0.184524,0.637527,0.220588,0.169832,0.330512,0.250000,0.651163,0.652720,0.013850
5,0.557471,0.583333,0.382530,0.463920,0.261985,0.723027,0.181818,0.167818,0.303571,0.773987,0.220588,0.167097,0.278953,0.333333,0.744186,0.667219,0.016620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20056,0.477011,0.250000,0.686747,0.587312,0.782917,0.254428,0.439394,0.196491,0.726190,0.170576,0.558824,0.194344,0.656791,0.750000,0.271318,0.109500,0.540166
20057,0.408046,0.083333,0.701807,0.729453,0.866475,0.162641,0.500000,0.194651,0.708333,0.211087,0.500000,0.188668,0.727203,0.583333,0.124031,0.366197,0.542936
20058,0.522989,0.500000,0.665663,0.684979,0.775321,0.175523,0.515152,0.198196,0.738095,0.281450,0.529412,0.212148,0.922278,0.833333,0.232558,0.053991,0.545706
20059,0.436782,0.750000,0.608434,0.746021,0.747468,0.133655,0.530303,0.233285,0.916667,0.208955,0.514706,0.203065,0.823394,0.583333,0.116279,0.234466,0.548476


In [ ]:
Y_train=[]
a=0
for i in y_train:
  if a%10==0:
   Y_train.append(i)
  a=a+1

In [ ]:
Xf_train=[]
for i in range(2006):
  Y1=np.zeros(shape=(10,17))
  for j in range(10):
    X1=np.array([])
    for k in range(17):
      X1=np.append(X1,train.iloc[10*i+j][k])
    Y1[j]=X1
        
  Xf_train.append(Y1)
    

In [ ]:
!pip install tsfel
import tsfel

In [ ]:
# Retrieves a pre-defined feature configuration file to extract all available features
cfg= tsfel.get_features_by_domain("statistical")

# Extract features
X = tsfel.time_series_features_extractor(cfg, Xf_train,header_names = np.array(['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']))

*** Feature extraction started ***


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Using default sampling frequency set in configuration file.
  """



*** Feature extraction finished ***


In [ ]:
X

,10_ECDF Percentile Count_0,10_ECDF Percentile Count_1,10_ECDF Percentile_0,10_ECDF Percentile_1,10_ECDF_0,10_ECDF_1,10_ECDF_2,10_ECDF_3,10_ECDF_4,10_ECDF_5,10_ECDF_6,10_ECDF_7,10_ECDF_8,10_ECDF_9,10_Histogram_0,10_Histogram_1,10_Histogram_2,10_Histogram_3,10_Histogram_4,10_Histogram_5,10_Histogram_6,10_Histogram_7,10_Histogram_8,10_Histogram_9,10_Interquartile range,10_Kurtosis,10_Max,10_Mean,10_Mean absolute deviation,10_Median,10_Median absolute deviation,10_Min,10_Root mean square,10_Skewness,10_Standard deviation,10_Variance,11_ECDF Percentile Count_0,11_ECDF Percentile Count_1,11_ECDF Percentile_0,11_ECDF Percentile_1,...,8_Root mean square,8_Skewness,8_Standard deviation,8_Variance,9_ECDF Percentile Count_0,9_ECDF Percentile Count_1,9_ECDF Percentile_0,9_ECDF Percentile_1,9_ECDF_0,9_ECDF_1,9_ECDF_2,9_ECDF_3,9_ECDF_4,9_ECDF_5,9_ECDF_6,9_ECDF_7,9_ECDF_8,9_ECDF_9,9_Histogram_0,9_Histogram_1,9_Histogram_2,9_Histogram_3,9_Histogram_4,9_Histogram_5,9_Histogram_6,9_Histogram_7,9_Histogram_8,9_Histogram_9,9_Interquartile range,9_Kurtosis,9_Max,9_Mean,9_Mean absolute deviation,9_Median,9_Median absolute deviation,9_Min,9_Root mean square,9_Skewness,9_Standard deviation,9_Variance
0,2.0,8.0,0.637527,0.795309,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,2.0,0.128998,-0.949157,0.889126,0.721109,0.081023,0.704691,0.068230,0.577825,0.726805,0.236459,0.090817,0.008248,2.0,8.0,0.205882,0.250000,...,0.130480,-0.175741,0.020933,0.000438,2.000000,8.000000,0.166667,0.255952,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.074405,-0.613848,0.303571,0.213690,0.047024,0.214286,0.038690,0.107143,0.220552,-0.257644,0.054584,0.002979
1,2.0,8.0,0.635394,0.720682,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,0.057569,-0.557355,0.812367,0.695949,0.050064,0.682303,0.033049,0.597015,0.698828,0.509821,0.063375,0.004016,2.0,8.0,0.205882,0.294118,...,0.132624,1.009512,0.020049,0.000402,2.000000,8.000000,0.178571,0.315476,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,0.087798,-1.134271,0.351190,0.250000,0.052381,0.232143,0.044643,0.160714,0.257237,0.281655,0.060586,0.003671
2,2.0,8.0,0.688699,0.793177,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,0.088486,-0.436653,0.827292,0.738380,0.053092,0.739872,0.047974,0.603412,0.741278,-0.411633,0.065483,0.004288,2.0,8.0,0.205882,0.279412,...,0.126011,0.814255,0.018905,0.000357,0.238095,0.238095,0.238095,0.238095,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.092262,-1.688839,0.351190,0.291071,0.042381,0.306548,0.035714,0.232143,0.294538,-0.198585,0.045054,0.002030
3,2.0,8.0,0.652452,0.767591,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.089019,-1.457362,0.795309,0.721962,0.047335,0.737740,0.043710,0.641791,0.723855,-0.224634,0.052317,0.002737,2.0,8.0,0.191176,0.294118,...,0.130716,0.102624,0.019332,0.000374,2.000000,8.000000,0.160714,0.285714,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,0.041667,-0.592176,0.380952,0.261905,0.052381,0.258929,0.023810,0.142857,0.271562,0.052360,0.071775,0.005152
4,2.0,8.0,0.648188,0.769723,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,0.083689,-0.056928,0.863539,0.711087,0.058422,0.698294,0.039446,0.571429,0.715111,0.220892,0.075753,0.005739,2.0,8.0,0.235294,0.294118,...,0.140256,0.232764,0.024444,0.000598,2.000000,8.000000,0.160714,0.357143,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,0.138393,-1.418234,0.369048,0.263095,0.075000,0.250000,0.083333,0.130952,0.275705,-0.103359,0.082427,0.006794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
test

,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,553.90,2388.04,9050.17,1.3,47.20,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,554.85,2388.01,9054.42,1.3,47.50,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,554.11,2388.05,9056.96,1.3,47.50,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,21.61,554.07,2388.03,9045.29,1.3,47.28,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,21.61,554.16,2388.01,9044.55,1.3,47.31,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,21.61,553.41,2388.02,9142.37,1.3,47.69,520.69,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,21.61,553.22,2388.05,9140.68,1.3,47.60,521.05,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,21.61,553.04,2388.11,9146.81,1.3,47.57,521.18,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,21.61,553.37,2388.07,9148.85,1.3,47.61,521.33,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699


In [ ]:
test['cycle_norm'] = test['time_cycles']
cols_normalize = test.columns.difference(['id','time_cycles','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test[cols_normalize]), columns=cols_normalize, index=test.index)
join_df = test[test.columns.difference(cols_normalize)].join(norm_test_df)
test = join_df.reindex(columns = test.columns)

test

In [ ]:
test = add_RUL(test)
test[index_names+['RUL']]

,unit_nr,time_cycles,RUL
0,0.0,1,30
1,0.0,2,29
2,0.0,3,28
3,0.0,4,27
4,0.0,5,26
...,...,...,...
13091,1.0,194,4
13092,1.0,195,3
13093,1.0,196,2
13094,1.0,197,1


In [ ]:
test.apply(pd.to_numeric)
list=[]
for i in range(0,13096):
  if test["RUL"][i]<50:
    list.append(1)
  elif test["RUL"][i]<100:
    list.append(2)
  elif test["RUL"][i]<150:
    list.append(3)
  else:
    list.append(4)

test["health"]=list

In [ ]:
test.drop('s_1', inplace=True, axis=1)
test.drop('s_5', inplace=True, axis=1)
test.drop('s_6', inplace=True, axis=1)
test.drop('s_10', inplace=True, axis=1)
test.drop('s_16', inplace=True, axis=1)
test.drop('s_18', inplace=True, axis=1)
test.drop('s_19', inplace=True, axis=1)
test.drop('unit_nr', inplace=True, axis=1)
test.drop('setting_3', inplace=True, axis=1)
test.drop('time_cycles', inplace=True, axis=1)

In [ ]:
curr_max = 30
test.apply(pd.to_numeric)
for i in range(0,13095):
  if test["RUL"][i]==0:
    curr_max = math.floor((test["RUL"][i+1])/10)*10
  if test["RUL"][i]>=curr_max:
    test.drop(i, inplace=True, axis=0)


test

,setting_1,setting_2,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21,cycle_norm,RUL,health
1,0.34375,0.230769,0.182965,0.504025,0.225240,0.800403,0.292683,0.229042,0.479452,0.634703,0.395349,0.277907,0.227709,0.500,0.645455,0.645718,0.003311,29,1
2,0.53125,0.538462,0.419558,0.464814,0.346130,0.651210,0.390244,0.248506,0.479452,0.591324,0.325581,0.192892,0.533557,0.500,0.700000,0.681104,0.006623,28,1
3,0.77500,0.461538,0.413249,0.391587,0.449867,0.643145,0.341463,0.159080,0.328767,0.456621,0.372093,0.217896,0.282359,0.250,0.627273,0.620382,0.009934,27,1
4,0.60000,0.461538,0.435331,0.471306,0.357974,0.661290,0.292683,0.153410,0.349315,0.632420,0.325581,0.187891,0.337009,0.125,0.618182,0.676008,0.013245,26,1
5,0.58750,0.692308,0.309148,0.261750,0.219318,0.673387,0.268293,0.202529,0.315068,0.579909,0.441860,0.169316,0.436242,0.375,0.545455,0.582166,0.016556,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,0.81875,0.461538,0.665615,0.789665,0.641209,0.510081,0.317073,0.902989,0.609589,0.299087,0.255814,0.935703,0.664909,0.625,0.309091,0.370842,0.639073,4,1
13092,0.44375,0.384615,0.659306,0.692028,0.769042,0.471774,0.390244,0.890038,0.547945,0.381279,0.465116,0.914003,0.567593,0.750,0.236364,0.483652,0.642384,3,1
13093,0.47500,0.230769,0.728707,0.626071,0.458036,0.435484,0.536585,0.937011,0.527397,0.410959,0.348837,0.971066,0.594919,0.750,0.281818,0.381741,0.645695,2,1
13094,0.27500,0.538462,0.671924,0.673851,0.714111,0.502016,0.439024,0.952644,0.554795,0.445205,0.441860,1.000000,0.662991,0.750,0.318182,0.473461,0.649007,1,1


In [ ]:
new_index = []
for i in range(1,12581):
  new_index.append(i)

In [ ]:
s = pd.Series(new_index)
test = test.set_index([s])

In [ ]:
for i in range(0,1258):
  curr = 0
  for j in range(1,11):
    curr = curr+(test["health"][(10*i)+j]) 
  curr = round(curr/10)
  for j in range(1,11):
    test["health"][(10*i)+j]=curr  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
test.drop('RUL', inplace=True, axis=1)

In [ ]:
y_test=test['health']
test.drop('health', inplace=True, axis=1)

In [ ]:
test

,setting_1,setting_2,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21,cycle_norm
1,0.34375,0.230769,0.182965,0.504025,0.225240,0.800403,0.292683,0.229042,0.479452,0.634703,0.395349,0.277907,0.227709,0.500,0.645455,0.645718,0.003311
2,0.53125,0.538462,0.419558,0.464814,0.346130,0.651210,0.390244,0.248506,0.479452,0.591324,0.325581,0.192892,0.533557,0.500,0.700000,0.681104,0.006623
3,0.77500,0.461538,0.413249,0.391587,0.449867,0.643145,0.341463,0.159080,0.328767,0.456621,0.372093,0.217896,0.282359,0.250,0.627273,0.620382,0.009934
4,0.60000,0.461538,0.435331,0.471306,0.357974,0.661290,0.292683,0.153410,0.349315,0.632420,0.325581,0.187891,0.337009,0.125,0.618182,0.676008,0.013245
5,0.58750,0.692308,0.309148,0.261750,0.219318,0.673387,0.268293,0.202529,0.315068,0.579909,0.441860,0.169316,0.436242,0.375,0.545455,0.582166,0.016556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12576,0.81875,0.461538,0.665615,0.789665,0.641209,0.510081,0.317073,0.902989,0.609589,0.299087,0.255814,0.935703,0.664909,0.625,0.309091,0.370842,0.639073
12577,0.44375,0.384615,0.659306,0.692028,0.769042,0.471774,0.390244,0.890038,0.547945,0.381279,0.465116,0.914003,0.567593,0.750,0.236364,0.483652,0.642384
12578,0.47500,0.230769,0.728707,0.626071,0.458036,0.435484,0.536585,0.937011,0.527397,0.410959,0.348837,0.971066,0.594919,0.750,0.281818,0.381741,0.645695
12579,0.27500,0.538462,0.671924,0.673851,0.714111,0.502016,0.439024,0.952644,0.554795,0.445205,0.441860,1.000000,0.662991,0.750,0.318182,0.473461,0.649007


In [ ]:
Y_test=[]
a=0
for i in y_test:
  if a%10==0:
   Y_test.append(i)
  a=a+1

In [ ]:
Xf_test=[]
for i in range(1258):
  Y1=np.zeros(shape=(10,17))
  for j in range(10):
    X1=np.array([])
    for k in range(17):
      X1=np.append(X1,test.iloc[10*i+j][k])
    Y1[j]=X1
        
  Xf_test.append(Y1)

In [ ]:
Xf_test

[array([[0.34375   , 0.23076923, 0.1829653 , 0.50402493, 0.22523994,
         0.80040323, 0.29268293, 0.22904215, 0.47945205, 0.6347032 ,
         0.39534884, 0.27790677, 0.22770853, 0.5       , 0.64545455,
         0.64571833, 0.00331126],
        [0.53125   , 0.53846154, 0.41955836, 0.46481433, 0.34613028,
         0.65120968, 0.3902439 , 0.24850575, 0.47945205, 0.5913242 ,
         0.3255814 , 0.19289159, 0.53355705, 0.5       , 0.7       ,
         0.68110403, 0.00662252],
        [0.775     , 0.46153846, 0.41324921, 0.3915866 , 0.44986727,
         0.64314516, 0.34146341, 0.15908046, 0.32876712, 0.456621  ,
         0.37209302, 0.21789605, 0.28235858, 0.25      , 0.62727273,
         0.62038217, 0.00993377],
        [0.6       , 0.46153846, 0.43533123, 0.47130615, 0.35797427,
         0.66129032, 0.29268293, 0.15340996, 0.34931507, 0.63242009,
         0.3255814 , 0.18789069, 0.33700863, 0.125     , 0.61818182,
         0.67600849, 0.01324503],
        [0.5875    , 0.69230769, 0.3

In [ ]:
X_test = tsfel.time_series_features_extractor(cfg, Xf_test,header_names = np.array(['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']))

*** Feature extraction started ***


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Using default sampling frequency set in configuration file.
  """Entry point for launching an IPython kernel.



*** Feature extraction finished ***


In [ ]:
X_test

,10_ECDF Percentile Count_0,10_ECDF Percentile Count_1,10_ECDF Percentile_0,10_ECDF Percentile_1,10_ECDF_0,10_ECDF_1,10_ECDF_2,10_ECDF_3,10_ECDF_4,10_ECDF_5,10_ECDF_6,10_ECDF_7,10_ECDF_8,10_ECDF_9,10_Histogram_0,10_Histogram_1,10_Histogram_2,10_Histogram_3,10_Histogram_4,10_Histogram_5,10_Histogram_6,10_Histogram_7,10_Histogram_8,10_Histogram_9,10_Interquartile range,10_Kurtosis,10_Max,10_Mean,10_Mean absolute deviation,10_Median,10_Median absolute deviation,10_Min,10_Root mean square,10_Skewness,10_Standard deviation,10_Variance,11_ECDF Percentile Count_0,11_ECDF Percentile Count_1,11_ECDF Percentile_0,11_ECDF Percentile_1,...,8_Root mean square,8_Skewness,8_Standard deviation,8_Variance,9_ECDF Percentile Count_0,9_ECDF Percentile Count_1,9_ECDF Percentile_0,9_ECDF Percentile_1,9_ECDF_0,9_ECDF_1,9_ECDF_2,9_ECDF_3,9_ECDF_4,9_ECDF_5,9_ECDF_6,9_ECDF_7,9_ECDF_8,9_ECDF_9,9_Histogram_0,9_Histogram_1,9_Histogram_2,9_Histogram_3,9_Histogram_4,9_Histogram_5,9_Histogram_6,9_Histogram_7,9_Histogram_8,9_Histogram_9,9_Interquartile range,9_Kurtosis,9_Max,9_Mean,9_Mean absolute deviation,9_Median,9_Median absolute deviation,9_Min,9_Root mean square,9_Skewness,9_Standard deviation,9_Variance
0,2.0,8.0,0.579909,0.618721,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.027397,3.241436,0.634703,0.590411,0.029589,0.599315,0.015982,0.456621,0.592349,-2.011032,0.047882,0.002293,0.395349,0.395349,0.395349,0.395349,...,0.203677,-0.321024,0.027861,0.000776,2.0,8.0,0.280822,0.390411,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,0.0,0.0,0.080479,-0.693906,0.479452,0.350000,0.059863,0.339041,0.047945,0.232877,0.358246,0.502774,0.076421,0.005840
1,2.0,8.0,0.586758,0.634703,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.030822,2.264922,0.682648,0.609361,0.036849,0.615297,0.017123,0.456621,0.612082,-1.570201,0.057654,0.003324,2.000000,8.000000,0.279070,0.395349,...,0.175708,0.183757,0.029967,0.000898,2.0,8.0,0.308219,0.404110,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,3.0,0.0,0.0,0.077055,-1.263034,0.452055,0.368493,0.044110,0.376712,0.054795,0.301370,0.372000,0.189424,0.050962,0.002597
2,2.0,8.0,0.547945,0.611872,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,3.0,0.0,0.056507,-0.124679,0.687215,0.582192,0.046119,0.574201,0.025114,0.463470,0.585208,-0.073869,0.059334,0.003521,2.000000,8.000000,0.348837,0.441860,...,0.191308,0.377003,0.035258,0.001243,2.0,8.0,0.267123,0.390411,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,2.0,0.0,0.0,0.085616,0.076817,0.424658,0.334247,0.057808,0.349315,0.047945,0.171233,0.341986,-0.862972,0.072344,0.005234
3,2.0,8.0,0.426941,0.504566,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,0.0,0.058790,-0.007735,0.632420,0.487215,0.050685,0.478311,0.033105,0.390411,0.491773,0.782311,0.066799,0.004462,2.000000,8.000000,0.418605,0.534884,...,0.174330,0.324712,0.028407,0.000807,2.0,8.0,0.356164,0.500000,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,0.106164,-1.197450,0.582192,0.452055,0.071233,0.434932,0.065068,0.342466,0.459272,0.316255,0.081100,0.006577
4,2.0,8.0,0.404110,0.520548,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,2.0,0.0,0.091895,-0.229726,0.625571,0.482877,0.075114,0.490868,0.053653,0.283105,0.492447,-0.364387,0.096612,0.009334,2.000000,8.000000,0.465116,0.534884,...,0.199910,-0.326004,0.024724,0.000611,2.0,8.0,0.390411,0.506849,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.058219,-0.602044,0.561644,0.468493,0.046575,0.476027,0.030822,0.363014,0.471887,-0.368315,0.056498,0.003192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import OneClassSVM

In [ ]:
clf1 =OneClassSVM()
clf2=DecisionTreeClassifier(criterion='entropy')
clf3=DecisionTreeClassifier()
clf4=SVC(gamma='auto')
clf5=GaussianProcessClassifier()
kernel=1.0*RBF(1.0)
clf6=GaussianProcessClassifier(kernel=kernel,random_state=0)
clf7=RandomForestClassifier()
clf8=ExtraTreesClassifier()
clf9=GaussianNB()
clf10=QuadraticDiscriminantAnalysis()


In [ ]:
X

,10_ECDF Percentile Count_0,10_ECDF Percentile Count_1,10_ECDF Percentile_0,10_ECDF Percentile_1,10_ECDF_0,10_ECDF_1,10_ECDF_2,10_ECDF_3,10_ECDF_4,10_ECDF_5,10_ECDF_6,10_ECDF_7,10_ECDF_8,10_ECDF_9,10_Histogram_0,10_Histogram_1,10_Histogram_2,10_Histogram_3,10_Histogram_4,10_Histogram_5,10_Histogram_6,10_Histogram_7,10_Histogram_8,10_Histogram_9,10_Interquartile range,10_Kurtosis,10_Max,10_Mean,10_Mean absolute deviation,10_Median,10_Median absolute deviation,10_Min,10_Root mean square,10_Skewness,10_Standard deviation,10_Variance,11_ECDF Percentile Count_0,11_ECDF Percentile Count_1,11_ECDF Percentile_0,11_ECDF Percentile_1,...,8_Root mean square,8_Skewness,8_Standard deviation,8_Variance,9_ECDF Percentile Count_0,9_ECDF Percentile Count_1,9_ECDF Percentile_0,9_ECDF Percentile_1,9_ECDF_0,9_ECDF_1,9_ECDF_2,9_ECDF_3,9_ECDF_4,9_ECDF_5,9_ECDF_6,9_ECDF_7,9_ECDF_8,9_ECDF_9,9_Histogram_0,9_Histogram_1,9_Histogram_2,9_Histogram_3,9_Histogram_4,9_Histogram_5,9_Histogram_6,9_Histogram_7,9_Histogram_8,9_Histogram_9,9_Interquartile range,9_Kurtosis,9_Max,9_Mean,9_Mean absolute deviation,9_Median,9_Median absolute deviation,9_Min,9_Root mean square,9_Skewness,9_Standard deviation,9_Variance
0,2.0,8.0,0.637527,0.795309,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,2.0,0.128998,-0.949157,0.889126,0.721109,0.081023,0.704691,0.068230,0.577825,0.726805,0.236459,0.090817,0.008248,2.0,8.0,0.205882,0.250000,...,0.130480,-0.175741,0.020933,0.000438,2.000000,8.000000,0.166667,0.255952,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.074405,-0.613848,0.303571,0.213690,0.047024,0.214286,0.038690,0.107143,0.220552,-0.257644,0.054584,0.002979
1,2.0,8.0,0.635394,0.720682,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,0.057569,-0.557355,0.812367,0.695949,0.050064,0.682303,0.033049,0.597015,0.698828,0.509821,0.063375,0.004016,2.0,8.0,0.205882,0.294118,...,0.132624,1.009512,0.020049,0.000402,2.000000,8.000000,0.178571,0.315476,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,0.087798,-1.134271,0.351190,0.250000,0.052381,0.232143,0.044643,0.160714,0.257237,0.281655,0.060586,0.003671
2,2.0,8.0,0.688699,0.793177,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,0.088486,-0.436653,0.827292,0.738380,0.053092,0.739872,0.047974,0.603412,0.741278,-0.411633,0.065483,0.004288,2.0,8.0,0.205882,0.279412,...,0.126011,0.814255,0.018905,0.000357,0.238095,0.238095,0.238095,0.238095,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.092262,-1.688839,0.351190,0.291071,0.042381,0.306548,0.035714,0.232143,0.294538,-0.198585,0.045054,0.002030
3,2.0,8.0,0.652452,0.767591,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.089019,-1.457362,0.795309,0.721962,0.047335,0.737740,0.043710,0.641791,0.723855,-0.224634,0.052317,0.002737,2.0,8.0,0.191176,0.294118,...,0.130716,0.102624,0.019332,0.000374,2.000000,8.000000,0.160714,0.285714,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,0.041667,-0.592176,0.380952,0.261905,0.052381,0.258929,0.023810,0.142857,0.271562,0.052360,0.071775,0.005152
4,2.0,8.0,0.648188,0.769723,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,0.083689,-0.056928,0.863539,0.711087,0.058422,0.698294,0.039446,0.571429,0.715111,0.220892,0.075753,0.005739,2.0,8.0,0.235294,0.294118,...,0.140256,0.232764,0.024444,0.000598,2.000000,8.000000,0.160714,0.357143,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,0.0,0.138393,-1.418234,0.369048,0.263095,0.075000,0.250000,0.083333,0.130952,0.275705,-0.103359,0.082427,0.006794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
X_train = X

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
fs = SelectKBest(score_func=f_classif, k=30)
fit = fs.fit(X_train,Y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Selected_columns','Score_ANOVA'] 
print(featureScores.nlargest(30,'Score_ANOVA'))  

         Selected_columns  Score_ANOVA
464    5_Root mean square  1242.848975
459                5_Mean  1235.221624
176   14_Root mean square  1235.070303
171               14_Mean  1231.343218
140   13_Root mean square  1222.946185
135               13_Mean  1219.721412
454         5_Histogram_8  1207.408286
428    4_Root mean square  1203.352946
423                4_Mean  1199.965460
243               16_Mean  1197.351945
248   16_Root mean square  1194.880725
608    9_Root mean square  1185.579799
461              5_Median  1184.039235
392    3_Root mean square  1182.091254
207               15_Mean  1181.828414
603                9_Mean  1180.304908
137             13_Median  1179.507533
212   15_Root mean square  1178.993990
387                3_Mean  1178.974833
435   5_ECDF Percentile_1  1166.168146
200        15_Histogram_6  1156.486262
236        16_Histogram_6  1155.824961
245             16_Median  1154.420128
605              9_Median  1149.273413
111  13_ECDF Percentile_1

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  8  13  14  15  16  17  18  44  49  50  51  52  53  54  80  85  86  87
  88  89  90 116 121 122 123 124 125 126 152 157 158 159 160 161 162 188
 193 194 195 196 197 198 224 229 230 231 232 233 234 252 253 260 265 266
 267 268 269 270 296 301 302 303 304 305 306 332 337 338 339 340 341 342
 368 373 374 375 376 377 378 404 409 410 411 412 413 414 440 445 446 447
 448 449 450 476 481 482 483 484 485 486 512 517 518 519 520 521 522 548
 553 554 555 556 557 558 584 589 590 591 592 593 594] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
liq=featureScores.nlargest(30,'Score_ANOVA')['Selected_columns']

In [ ]:
X_train1 = X_train.filter(['10_ECDF Percentile_0','12_Median','10_Variance','11_ECDF Percentile_0','16_Median','11_ECDF Percentile Count_0','16_Mean','16_Standard deviation','10_Kurtosis','10_Min','16_ECDF Percentile_1','10_Standard deviation','10_ECDF Percentile_1','16_Max','16_Variance','16_Root mean square','10_Max','12_Min','10_Median','11_Variance','16_Min','15_Kurtosis','11_Mean absolute deviation','10_Root mean square','15_Variance','10_Mean absolute deviation','11_Median absolute deviation','16_Interquartile range','12_Kurtosis','13_ECDF Percentile_0'], axis=1)

In [ ]:
X_train1

,10_ECDF Percentile_0,12_Median,10_Variance,11_ECDF Percentile_0,16_Median,11_ECDF Percentile Count_0,16_Mean,16_Standard deviation,10_Kurtosis,10_Min,16_ECDF Percentile_1,10_Standard deviation,10_ECDF Percentile_1,16_Max,16_Variance,16_Root mean square,10_Max,12_Min,10_Median,11_Variance,16_Min,15_Kurtosis,11_Mean absolute deviation,10_Root mean square,15_Variance,10_Mean absolute deviation,11_Median absolute deviation,16_Interquartile range,12_Kurtosis,13_ECDF Percentile_0
0,0.637527,0.170812,0.008248,0.205882,0.664803,2.0,0.684355,0.068465,-0.949157,0.577825,0.707539,0.090817,0.795309,0.807097,0.004687,0.687771,0.889126,0.132883,0.704691,0.000813,0.574979,-0.946779,0.022941,0.726805,0.002738,0.081023,0.014706,0.054992,0.777107,0.184302
1,0.635394,0.163226,0.004016,0.205882,0.649544,2.0,0.650318,0.069935,-0.557355,0.597015,0.696631,0.063375,0.720682,0.774372,0.004891,0.654067,0.812367,0.142533,0.682303,0.001687,0.526788,-0.619052,0.038235,0.698828,0.007613,0.050064,0.036765,0.081676,-1.362404,0.275106
2,0.688699,0.165987,0.004288,0.205882,0.716308,2.0,0.699351,0.083653,-0.436653,0.603412,0.763601,0.065483,0.793177,0.836371,0.006998,0.704336,0.827292,0.130767,0.739872,0.002474,0.538387,-0.407554,0.036471,0.741278,0.001664,0.053092,0.022059,0.111054,-1.192300,0.257022
3,0.652452,0.166555,0.002737,0.191176,0.693800,2.0,0.696382,0.066300,-1.457362,0.641791,0.747031,0.052317,0.767591,0.804474,0.004396,0.699531,0.795309,0.137630,0.737740,0.001548,0.573046,-1.248324,0.032353,0.723855,0.011745,0.047335,0.036765,0.086889,-0.593696,0.205848
4,0.648188,0.151770,0.005739,0.235294,0.676885,2.0,0.695043,0.075573,-0.056928,0.571429,0.767191,0.075753,0.769723,0.818973,0.005711,0.699139,0.863539,0.135566,0.698294,0.001419,0.580779,-0.432221,0.029412,0.715111,0.003957,0.058422,0.022059,0.103528,-1.189889,0.280108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,0.405117,0.231190,0.005532,0.352941,0.433168,2.0,0.437545,0.093388,-0.722769,0.373134,0.536178,0.074377,0.526652,0.586026,0.008721,0.447400,0.620469,0.178759,0.453092,0.001395,0.297294,-1.370633,0.029412,0.478523,0.005634,0.063198,0.022059,0.142847,-0.223695,0.460562
2002,0.292111,0.227861,0.006959,0.367647,0.361433,2.0,0.378528,0.094750,-1.501653,0.281450,0.460232,0.083419,0.469083,0.510080,0.008978,0.390206,0.511727,0.195325,0.361407,0.001678,0.222176,-0.802277,0.035294,0.401929,0.009389,0.076930,0.029412,0.156034,-1.475590,0.495575
2003,0.326226,0.216870,0.006864,0.411765,0.385598,2.0,0.358451,0.085253,-0.688733,0.238806,0.411489,0.082848,0.449893,0.469898,0.007268,0.368449,0.528785,0.199401,0.379531,0.000735,0.194698,-0.560396,0.020588,0.402645,0.009339,0.068998,0.014706,0.103079,-1.380382,0.616391
2004,0.176972,0.210238,0.007964,0.470588,0.311930,2.0,0.316694,0.085139,-1.124167,0.172708,0.391052,0.089241,0.415778,0.464789,0.007249,0.327939,0.430704,0.196821,0.300640,0.001083,0.222314,-1.138799,0.027059,0.327531,0.005962,0.074968,0.022059,0.141294,-0.710314,0.598307


In [ ]:
X_test1 = X_test.filter(['10_ECDF Percentile_0','12_Median','10_Variance','11_ECDF Percentile_0','16_Median','11_ECDF Percentile Count_0','16_Mean','16_Standard deviation','10_Kurtosis','10_Min','16_ECDF Percentile_1','10_Standard deviation','10_ECDF Percentile_1','16_Max','16_Variance','16_Root mean square','10_Max','12_Min','10_Median','11_Variance','16_Min','15_Kurtosis','11_Mean absolute deviation','10_Root mean square','15_Variance','10_Mean absolute deviation','11_Median absolute deviation','16_Interquartile range','12_Kurtosis','13_ECDF Percentile_0'], axis=1)

In [ ]:
clf2.fit(X_train1,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
Y_pred_2 = clf2.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_2)

0.383147853736089

In [ ]:
clf3.fit(X_train1,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
Y_pred_3 = clf3.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_3)

0.3386327503974563

In [ ]:
clf4.fit(X_train1,Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
Y_pred_4 = clf4.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_4)

0.390302066772655

In [ ]:
clf5.fit(X_train1,Y_train)

GaussianProcessClassifier(copy_X_train=True, kernel=None, max_iter_predict=100,
                          multi_class='one_vs_rest', n_jobs=None,
                          n_restarts_optimizer=0, optimizer='fmin_l_bfgs_b',
                          random_state=None, warm_start=False)

In [ ]:
Y_pred_5 = clf5.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_5)

0.4205087440381558

In [ ]:
clf6.fit(X_train1,Y_train)

GaussianProcessClassifier(copy_X_train=True, kernel=1**2 * RBF(length_scale=1),
                          max_iter_predict=100, multi_class='one_vs_rest',
                          n_jobs=None, n_restarts_optimizer=0,
                          optimizer='fmin_l_bfgs_b', random_state=0,
                          warm_start=False)

In [ ]:
Y_pred_6 = clf6.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_6)

0.41414944356120825

In [ ]:
clf7.fit(X_train1,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
Y_pred_7 = clf7.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_7)

0.3958664546899841

In [ ]:
clf8.fit(X_train1,Y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [ ]:
Y_pred_8 = clf8.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_8)

0.3990461049284579

In [ ]:
clf9.fit(X_train1,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
Y_pred_9 = clf9.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_9)

0.4189189189189189

In [ ]:
clf10.fit(X_train1,Y_train)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001)

In [ ]:
Y_pred_10 = clf10.predict(X_test1)

In [ ]:
metrics.accuracy_score(Y_test,Y_pred_10)

0.4133545310015898